# Classification, Segmentation and Detection of tumors in mammography medical images (mejorar)

## Authors:
- **Sebastián Kay Conde Lorenzo (correo aquí)**
- **Jaime Capdepon Fraile (correo aquí)**
- **Joaquín Negrete Saab (correo aquí)**
- **Yang Liu (correo aquí)**
- **Christian Most Tazon (correo aquí)**
  
## Summary:
aquí una descripción

---



## Imports

In [1]:
# Third library imports
import typing
import numpy as np
import cv2 as cv
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os


# Custom imports
import dir_manager
import viewer
import models

## Data download

TODO: Aquí deberíamos poner que los datos se descarguen de una API o bien se saquen de una BBDD (a discutir)

In [2]:
#

## Directory tree creation

In [3]:
#manager = dir_manager.DirectoryManager("../dataset")
#manager.manage_directories()

## Dataset visualization

TODO: EL SINGLE VIEWER DEBERIA TOMAR UNA RUTA A UN DIRECTORIO

TODO: EL SINGLE VIEWER TIENE QUE ACEPTAR LA IMAGEN CON FORMATO (.jpg) NO ASUMIRLO (ASI ES MENOS TEDIOSO)

In [4]:
single_viewer = viewer.SingleViewer(file_name="0_DBT-P00013_36_png.rf.ac6715e066245d20fac35a0033c2cdcb", dataset="train", title="Single Visualization")
#single_viewer.plot_img()
#single_viewer.plot_bounding_box()

In [5]:
all_train_images_filenames : list[str] = os.listdir("../dataset/train/images")
all_train_images_filenames = [name[:-4] for name in all_train_images_filenames] # Para eliminar el ".jpg"

In [6]:
# Visualizacion de 10 imagenes aleatorias
batch_viewer = viewer.BatchViewer(file_names=all_train_images_filenames[:10],
                                  dataset="train",
                                  titles=["Batch Visualization"]*len(all_train_images_filenames[:10]))
#batch_viewer.plot_imgs()
#batch_viewer.plot_bounding_boxes()

nota: el tamano de las imagenes es de 640

## Model Creation

In [7]:
resnet50 : models.ClassificationModel = models.ClassificationModel(1, 2, [3,4,6,3]) 

In [8]:
resnet50

ClassificationModel(
  (conv1_layer): Sequential(
    (conv1_1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm1_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (ReLU1): ReLU()
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2_layer): Sequential(
    (block2_1): ResNetBlock(
      (block): Sequential(
        (conv2_1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm2_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batchnorm2_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2_3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm2_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    

In [9]:
# Input de prueba
BATCH_SIZE = 4
x : torch.tensor = torch.randn(BATCH_SIZE, 1, 224, 224)
y : torch.tensor = resnet50(x)

print(y.shape)

torch.Size([4, 2])
